In [ ]:
!pip install pandas
!pip install matplotlib
!pip install scipy

In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.stats import norm

In [ ]:
class colors:
    WHITE = '\033[97m'
    BLACK = '\033[90m'
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    CYAN = '\033[96m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
def blue(text):
    return colors.BLUE+text+colors.BLACK

def green(text):
    return colors.GREEN+text+colors.BLACK

def red(text):
    return colors.RED+text+colors.BLACK

def yellow(text):
    return colors.YELLOW+text+colors.BLACK

def load_run(runname):
    rundata=pd.read_csv(runname+'.csv')
    
    with open(runname+'_report.json') as f:
        runinfo=json.load(f)

    if 'clock_freq' in runinfo:
        clock_freq=runinfo['clock_freq']
    else:
        clock_freq=None

    if 'clock_cycles' in rundata:
        clock_mean=rundata['clock_cycles'].mean()
        clock_std=rundata['clock_cycles'].std()
        inference_time_mean=clock_mean/(clock_freq) 
    else:
        clock_mean=None
        clock_std=None
        inference_time_mean=None
        
    runinfo["runname"]=runname
    runinfo["rundata"]=rundata
    runinfo["clock_freq"]=clock_freq
    runinfo["clock_mean"]=clock_mean
    runinfo["clock_std"]=clock_std
    runinfo["inference_time_mean"]=inference_time_mean
    
    return runinfo

In [ ]:
def consistency(runs, pivot_run):
    # Result consistency, compare to pivot run every other run by mean and std
    for i in range(len(runs)):
        if i!= pivot_run:
            runname=runs[i]["runname"]
            rundata0=abs(runs[i]["rundata"]["probability_0"]-runs[pivot_run]["rundata"]["probability_0"])
            rundata0_mean=rundata0.mean()
            rundata0_std=rundata0.std()
            rundata1=abs(runs[i]["rundata"]["probability_1"]-runs[pivot_run]["rundata"]["probability_1"])
            rundata1_mean=rundata1.mean()
            rundata1_std=rundata1.std()
            prediction=runs[i]["rundata"]["classification"]==runs[pivot_run]["rundata"]["classification"]
           
            print (blue(runname)+" - probability_0 - mean: "+str(rundata0_mean)+ " std: "+str(rundata0_std))
            print (blue(runname)+" - probability_1 - mean: "+str(rundata1_mean)+ " std: "+str(rundata1_std))
            print (blue(runname)+" - prediction    - "+ green(str(prediction.value_counts(normalize=True)[True]*100)+ "%")+" equal")
            print ()

In [ ]:
def writetime(runs, pivot_run, x_param, y_param, title):
    x=[]
    y=[]
    names=[]
    for i in range(len(runs)):
        if x_param in runs[i] and y_param in runs[i]:
            x.append(runs[i][x_param])
            y.append(runs[i][y_param])
            names.append(runs[i]["runname"])

            print(runs[i]["runname"], runs[i][x_param], runs[i][y_param])

    if len(names) < 2:
        print ("Not enought data to plot")
        return

In [ ]:
def analyze(runs, pivot_run):
    pivot_name=runs[pivot_run]["runname"]
    print ("Pivot run "+  blue(pivot_name))
    print ()
    print (red("Consistency, compare to pivot run every other run"))
    consistency(runs, pivot_run)
    
    print (red("Inference time mean"))
    writetime(runs, pivot_run, "expprec", "inference_time_mean", "")

    print (red("Occupancy"))
    writetime(runs, pivot_run, "cps", "luts", "Occupancy (cps vs luts)")

    print (red("Power consumption"))
    writetime(runs, pivot_run, "cps", "power", "Power consumption (cps vs power)")


In [ ]:
soft=load_run("sw")

In [ ]:
#expanded=load_run("expanded")
expprec2_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_11bit")
expprec3_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec3_flopoco_11bit")
expprec5_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_11bit")
expprec8_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec8_flopoco_11bit")
expprec10_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_11bit")
expprec13_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec13_flopoco_11bit")
expprec16_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec16_flopoco_11bit")
expprec20_flopoco_11bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_11bit")
print(expprec2_flopoco_11bit.keys())
print('clock_mean' , 'clock_std',\
          'luts', 'power', 'inference_time_mean', 'cps',)
for v in [expprec2_flopoco_11bit,expprec3_flopoco_11bit,\
          expprec5_flopoco_11bit,expprec8_flopoco_11bit,\
          expprec10_flopoco_11bit,expprec13_flopoco_11bit,\
          expprec16_flopoco_11bit,expprec20_flopoco_11bit]:
    print(v['clock_mean'] , v['clock_std'], \
          v['luts'], v['power'], v['inference_time_mean'], v['cps'],)

analyze([ soft, expprec2_flopoco_11bit, \
                expprec3_flopoco_11bit, \
                expprec5_flopoco_11bit, \
                expprec8_flopoco_11bit, \
                expprec10_flopoco_11bit, \
                expprec13_flopoco_11bit, \
                expprec16_flopoco_11bit, \
                expprec20_flopoco_11bit],0)

Clearly using few bit as soon as we start increasing the series expansion the results explode (we need more bits)
the time simply has a big  uncertainty

In [ ]:
#expanded=load_run("expanded")
expprec2_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_16bit")
expprec3_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec3_flopoco_16bit")
expprec5_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_16bit")
expprec8_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec8_flopoco_16bit")
expprec10_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_16bit")
expprec13_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec13_flopoco_16bit")
expprec16_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec16_flopoco_16bit")
expprec20_flopoco_16bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_16bit")

for v in [expprec2_flopoco_16bit,expprec3_flopoco_16bit,\
          expprec5_flopoco_16bit,expprec8_flopoco_16bit,\
          expprec10_flopoco_16bit, expprec13_flopoco_16bit, \
          expprec16_flopoco_16bit, expprec20_flopoco_16bit]:
    print(v['clock_mean'] , v['clock_std'], v['cps'], v['luts'], v['power'], v['inference_time_mean'])

analyze([ soft, expprec2_flopoco_16bit, \
                expprec3_flopoco_16bit, \
                expprec5_flopoco_16bit, \
                expprec8_flopoco_16bit, \
                expprec10_flopoco_16bit, \
                expprec20_flopoco_16bit],0)

Clearly using few bit as soon as we start increasing the series expansion the results explode (we need more bits)
as for 11 but ony for 20 

In [ ]:
#expanded=load_run("expanded")
expprec2_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_19bit")
expprec3_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec3_flopoco_19bit")
expprec5_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_19bit")
expprec8_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec8_flopoco_19bit")
expprec10_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_19bit")
expprec13_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec13_flopoco_19bit")
expprec16_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec16_flopoco_19bit")
expprec20_flopoco_19bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_19bit")

for v in [expprec2_flopoco_19bit,expprec3_flopoco_19bit,\
          expprec5_flopoco_19bit,expprec8_flopoco_19bit,\
          expprec10_flopoco_19bit,expprec13_flopoco_19bit,\
          expprec16_flopoco_19bit,expprec20_flopoco_19bit]:
    print(v['clock_mean'] , v['clock_std'], v['cps'], v['luts'], v['power'], v['inference_time_mean'])

analyze([ soft, expprec2_flopoco_19bit, \
                expprec3_flopoco_19bit, \
                expprec5_flopoco_19bit, \
                expprec8_flopoco_19bit, \
                expprec10_flopoco_19bit, \
                expprec13_flopoco_19bit, \
                expprec16_flopoco_19bit, \
                expprec20_flopoco_19bit],0)

In [ ]:
#expanded=load_run("expanded")
expprec2_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec2_flopoco_32bit")
expprec3_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec3_flopoco_32bit")
expprec5_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec5_flopoco_32bit")
expprec8_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec8_flopoco_32bit")
expprec10_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec10_flopoco_32bit")
expprec13_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec13_flopoco_32bit")
expprec16_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec16_flopoco_32bit")
expprec20_flopoco_32bit=load_run("proj_zedboard_diverge_mm_hwopt_expprec20_flopoco_32bit")

for v in [expprec2_flopoco_32bit,expprec3_flopoco_32bit,\
          expprec5_flopoco_32bit,expprec8_flopoco_32bit,\
          expprec10_flopoco_32bit,expprec13_flopoco_32bit,\
          expprec16_flopoco_32bit,expprec20_flopoco_32bit]:
    print(v['clock_mean'] , v['clock_std'], v['cps'], v['luts'], v['power'], v['inference_time_mean'])

analyze([ soft, expprec2_flopoco_32bit, \
                expprec3_flopoco_32bit, \
                expprec5_flopoco_32bit, \
                expprec8_flopoco_32bit, \
                expprec10_flopoco_32bit, \
                expprec13_flopoco_32bit, \
                expprec16_flopoco_32bit, \
                expprec20_flopoco_32bit],0)